In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
import validators

In [64]:
from langchain_groq import ChatGroq
groq_api = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api, model="llama3-70b-8192")

In [97]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_document(url):
    if validators.url(url):
        if 'youtube' in url or 'youtu.be' in url :
            loader = YoutubeLoader.from_youtube_url(url)
        else : 
            #loader = UnstructuredURLLoader(urls=[url], ssl_verify= False, headers = headers)
            loader = WebBaseLoader(web_path=url)
        return loader.load()
    else :
        

        print('Invalid URL')



def document_splitter(document , chunk_size = 5000, chunk_overlap = 500):
    splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
    return splitter.split_documents(document)

In [98]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

In [99]:
seo_template = """
You're an AI SEO expert writing articles to rank my blog. Create a comprehensive article . 

Follow these instructions:

1. Format the headings as HTML (H1, H2, H3, etc.).
3. Insert relevant external links to reputable sources throughout the content. Use at least 5 different sources.
4. Include the target keyword naturally in the first and last sentences of the article.
5. Optimize the content for the given keyword throughout the article.
6. Humanize the output:
   - Use a conversational and natural tone
   - Incorporate analogies and examples where appropriate
   - Add personal anecdotes or experiences (hypothetical if necessary)
   - Use rhetorical questions to engage the reader
   - Vary sentence structure and length for better readability

7. No fluffy AI jargon or generic buzzwords. Keep it neutral and conversational.
8. Create a compelling introduction that hooks the reader.
9. Include bullet points or numbered lists where appropriate for easy readability.
10. Add relevant FAQs at the end of the article.
11. Conclude with a strong summary that reinforces the main points.

use this for artical :  {text}
"""

seo_prompt = PromptTemplate(input_variable = ["text", "word_count"], template = template)



In [105]:
word_count = 1000
document = get_document("https://www.linkedin.com/jobs/view/3942953083")

In [107]:
job_template = """
Extract all the key information from this text, which can be used for job posting  : 

{text} , 

"""

job_prompt = PromptTemplate(input_variable = ["text"], template = job_template)

In [108]:
document

[Document(metadata={'source': 'https://www.linkedin.com/jobs/view/3942953083', 'title': 'BMW Group hiring Working student algorithmic development for Generative Design (f/m/x) in Munich, Bavaria, Germany | LinkedIn', 'description': 'Posted 9:10:17 AM. Some It Works. Some Changes What&#39;s Possible.SHARE YOUR PASSION.More than 90% of automotive…See this and similar jobs on LinkedIn.', 'language': 'en'}, page_content="\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBMW Group hiring Working student algorithmic development for Generative Design (f/m/x) in Munich, Bavaria, Germany | LinkedIn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\nLinkedIn respects your privacy\n\n\nLinkedIn and 3rd parties use essential and non-essential cookies to provide, secure, analyze and improve our Services, and to show you relevant ads (including professional and job ads) on and off LinkedIn. Learn more in our Cookie Policy.Select Accept to consent or Reject to decline non-ess

In [109]:
llm.get_num_tokens(document[0].page_content)

19250

In [110]:
llm_chain = LLMChain(llm=llm, prompt= job_prompt)
llm_chain.run({'text' : document})

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01j3dwkt7re4w9h0m2b2970xpn` on tokens per minute (TPM): Limit 6000, Used 0, Requested 8442. Please try again in 24.42s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [111]:
template = """
Please provide the summary of this  : 

{text} , 



"""

prompt = PromptTemplate(input_variable = ["text"], template = template)

In [112]:
template = """
Please provide the summary of this  : 

{text} , 



"""

prompt = PromptTemplate(input_variable = ["text"], template = template)

In [113]:
final_template = """ 
Provide final summary,   add bullet points, 

Text : {text}
"""

final_prompt = PromptTemplate(input_variables=['text'],template=final_template)


In [114]:
documents = document_splitter(document)

In [115]:
document[0].metadata['title']

'BMW Group hiring Working student algorithmic development for Generative Design (f/m/x) in Munich, Bavaria, Germany | LinkedIn'

In [116]:
documents

[Document(metadata={'source': 'https://www.linkedin.com/jobs/view/3942953083', 'title': 'BMW Group hiring Working student algorithmic development for Generative Design (f/m/x) in Munich, Bavaria, Germany | LinkedIn', 'description': 'Posted 9:10:17 AM. Some It Works. Some Changes What&#39;s Possible.SHARE YOUR PASSION.More than 90% of automotive…See this and similar jobs on LinkedIn.', 'language': 'en'}, page_content='BMW Group hiring Working student algorithmic development for Generative Design (f/m/x) in Munich, Bavaria, Germany | LinkedIn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\nLinkedIn respects your privacy\n\n\nLinkedIn and 3rd parties use essential and non-essential cookies to provide, secure, analyze and improve our Services, and to show you relevant ads (including professional and job ads) on and off LinkedIn. Learn more in our Cookie Policy.Select Accept to consent or Reject to decline non-essential cookies for this use. You can upda

In [117]:
from langchain.chains.summarize import load_summarize_chain
map_reduce_chain = load_summarize_chain(llm=llm , chain_type='map_reduce', map_prompt= prompt , combine_prompt = final_prompt )
final_summary = map_reduce_chain.run(documents)

In [118]:

llm_chain.run({'text' : final_summary, 'word_count':5000})

'Here is the extracted key information that can be used for a job posting:\n\n**Job Title:** Working Student Algorithmic Development for Generative Design\n\n**Company:** BMW Group\n\n**Location:** Munich, Bavaria, Germany\n\n**Job Type:** Student Staff/Working Student\n\n**Job Summary:**\n\nContribute to the development of generative design algorithms and tools for various applications within the BMW Group.\n\n**Responsibilities:**\n\n* Support the development of methods and AI methods for Generative Design\n* Implement innovative design tools for parametric and generative/algorithmic form-finding\n* Collaborate with the Generative Design Team to develop innovative design tools\n\n**Requirements:**\n\n* Degree in computer science, engineering, architecture, or a comparable qualification\n* Strong knowledge of Grasshopper and/or Synera\n* Strong knowledge of AI, including computer vision, LLM, and deep learning for simulation\n* Python and/or C# programming skills\n* Independent, solut

In [119]:
from langchain_core.messages import HumanMessage, AIMessage , SystemMessage

chat_message = [
    SystemMessage(content="You're an AI SEO expert writing articles. Create a comprehensive article . Make it 2000 words long"),
    HumanMessage(content=f"""
     use this for artical :  {final_summary}
     End of Text 
Follow these instructions:
Article title : {document[0].metadata['title']}
1. Include different headings.
3. Insert relevant external links to reputable sources throughout the content. Use at least 5 different sources.
4. Include the target keyword naturally in the first and last sentences of the article.
6. Humanize the output:
   - Use a conversational and natural tone
   - Incorporate analogies and examples where appropriate
   - Add personal anecdotes or experiences (hypothetical if necessary)
   - Use rhetorical questions to engage the reader
   - Vary sentence structure and length for better readability

7. No fluffy AI jargon or generic buzzwords. Keep it neutral and conversational.
8. Create a compelling introduction that hooks the reader.
9. Include bullet points or numbered lists where appropriate for easy readability.
10. Add relevant FAQs at the end of the article.
11. Conclude with a strong summary that reinforces the main points.
""")
]


In [120]:
output = llm(chat_message)

In [121]:
output.content

'**BMW Group hiring Working student algorithmic development for Generative Design (f/m/x) in Munich, Bavaria, Germany | LinkedIn**\n\nAre you a student with a passion for algorithmic development and generative design? Do you dream of working with a renowned company like BMW Group? If so, this might be the perfect opportunity for you! BMW Group is currently hiring a Working Student Algorithmic Development for Generative Design (f/m/x) to join their team in Munich, Bavaria, Germany.\n\n**What is Generative Design?**\n\nBefore we dive into the job description, let\'s take a step back and explore what generative design is. According to [Autodesk](https://www.autodesk.com/solutions/generative-design), "Generative design is a design methodology that involves using algorithms and artificial intelligence (AI) to generate multiple design options based on a set of inputs and constraints." This innovative approach allows designers to explore numerous possibilities and find the most optimal soluti

In [122]:
format_chat_message = [
    SystemMessage(content="You're an Text Converter"),
    HumanMessage(content=f"""
    
format the following text into html , with html tags. Remove all eol , escape , newline characters ,  * and \ ,/ :
                 

                 Text :  {output.content}
""")
]


In [123]:
formatted_output = llm(format_chat_message)

In [124]:
formatted_output.content.replace("\n", "")

'Here is the formatted text in HTML:<span>BMW Group hiring Working student algorithmic development for Generative Design (f/m/x) in Munich, Bavaria, Germany | LinkedIn</span><p>Are you a student with a passion for algorithmic development and generative design? Do you dream of working with a renowned company like BMW Group? If so, this might be the perfect opportunity for you! BMW Group is currently hiring a Working Student Algorithmic Development for Generative Design (f/m/x) to join their team in Munich, Bavaria, Germany.</p><h2>What is Generative Design?</h2><p>Before we dive into the job description, let\'s take a step back and explore what generative design is. According to <a href="https://www.autodesk.com/solutions/generative-design">Autodesk</a>, "Generative design is a design methodology that involves using algorithms and artificial intelligence (AI) to generate multiple design options based on a set of inputs and constraints." This innovative approach allows designers to explo